## PySpark SQL Date and Timestamp Functions
PySpark Date and Timestamp Functions are supported on DataFrame and SQL queries and they work similarly to traditional SQL, Date and Time are very important if you are using PySpark for ETL. Most of all these functions accept input as, Date type, Timestamp type, or String. If a String used, it should be in a default format that can be cast to date.

* DateType default format is yyyy-MM-dd 
* TimestampType default format is yyyy-MM-dd HH:mm:ss.SSSS
* Returns null if the input is a string that can not be cast to Date or Timestamp.

PySpark SQL provides several Date & Timestamp functions hence keep an eye on and understand these. Always you should choose these functions instead of writing your own functions (UDF) as these functions are compile-time safe, handles null, and perform better when compared to PySpark UDF. If your PySpark application is critical on performance try to avoid using custom UDF at all costs as these are not guarantee performance.

For readable purposes, I’ve grouped these functions into the following groups.
* Date Functions
* Timestamp Functions
* Date and Timestamp Window Functions

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

In [2]:
from pyspark.sql.functions import *

data=[["1","2020-02-01"],["2","2019-03-11"],["3","2021-03-28"]]
df=spark.createDataFrame(data,["id","input"])
df.show()

+---+----------+
| id|     input|
+---+----------+
|  1|2020-02-01|
|  2|2019-03-11|
|  3|2021-03-28|
+---+----------+



### PySpark SQL Date and Timestamp Functions
#### current_date()
Use current_date() to get the current system date. By default, the data will be returned in yyyy-dd-mm format.

In [3]:
#current_date()
df.select(current_date().alias("current_date")
  ).show(1)

+------------+
|current_date|
+------------+
|  2023-03-07|
+------------+
only showing top 1 row



#### date_format()
The below example uses date_format() to parses the date and converts from yyyy-dd-mm to MM-dd-yyyy format.

In [4]:
#date_format()
df2 = df.select(col("input"), 
    date_format(col("input"), "MM-dd-yyyy").alias("date_format") 
  )
df2.printSchema()
df2.show()

root
 |-- input: string (nullable = true)
 |-- date_format: string (nullable = true)

+----------+-----------+
|     input|date_format|
+----------+-----------+
|2020-02-01| 02-01-2020|
|2019-03-11| 03-11-2019|
|2021-03-28| 03-28-2021|
+----------+-----------+



#### to_date()
Below example converts string in date format yyyy-MM-dd to a DateType yyyy-MM-dd using to_date(). You can also use this to convert into any specific format. PySpark supports all patterns supports on Java DateTimeFormatter.

In [5]:
#to_date()
df2 = df.select(col("input"), 
    to_date(col("input"), "yyy-MM-dd").alias("to_date") 
  )
df2.printSchema()
df2.show()

root
 |-- input: string (nullable = true)
 |-- to_date: date (nullable = true)

+----------+----------+
|     input|   to_date|
+----------+----------+
|2020-02-01|2020-02-01|
|2019-03-11|2019-03-11|
|2021-03-28|2021-03-28|
+----------+----------+



#### datediff()
The below example returns the difference between two dates using datediff().

In [6]:
#datediff()
df.select(col("input"), 
    datediff(current_date(),col("input")).alias("datediff")  
  ).show()

+----------+--------+
|     input|datediff|
+----------+--------+
|2020-02-01|    1130|
|2019-03-11|    1457|
|2021-03-28|     709|
+----------+--------+



#### months_between()
The below example returns the months between two dates using months_between().

In [7]:
#months_between()
df.select(col("input"), 
    months_between(current_date(),col("input")).alias("months_between")  
  ).show()

+----------+--------------+
|     input|months_between|
+----------+--------------+
|2020-02-01|   37.19354839|
|2019-03-11|   47.87096774|
|2021-03-28|   23.32258065|
+----------+--------------+



#### trunc()
The below example truncates the date at a specified unit using trunc().

In [8]:
#trunc()
df.select(col("input"), 
    trunc(col("input"),"Month").alias("Month_Trunc"), 
    trunc(col("input"),"Year").alias("Month_Year")
   ).show()

+----------+-----------+----------+
|     input|Month_Trunc|Month_Year|
+----------+-----------+----------+
|2020-02-01| 2020-02-01|2020-01-01|
|2019-03-11| 2019-03-01|2019-01-01|
|2021-03-28| 2021-03-01|2021-01-01|
+----------+-----------+----------+



#### add_months() , date_add(), date_sub()
Here we are adding and subtracting date and month from a given input.

In [9]:
#add_months() , date_add(), date_sub()
df.select(col("input"), 
    add_months(col("input"),3).alias("add_months"), 
    add_months(col("input"),-3).alias("sub_months"), 
    date_add(col("input"),4).alias("date_add"), 
    date_sub(col("input"),4).alias("date_sub") 
  ).show()

+----------+----------+----------+----------+----------+
|     input|add_months|sub_months|  date_add|  date_sub|
+----------+----------+----------+----------+----------+
|2020-02-01|2020-05-01|2019-11-01|2020-02-05|2020-01-28|
|2019-03-11|2019-06-11|2018-12-11|2019-03-15|2019-03-07|
|2021-03-28|2021-06-28|2020-12-28|2021-04-01|2021-03-24|
+----------+----------+----------+----------+----------+



#### year(), month(), next_day(), weekofyear()

In [10]:
df.select(col("input"), 
     year(col("input")).alias("year"), 
     month(col("input")).alias("month"), 
     next_day(col("input"),"Sunday").alias("next_day"), 
     weekofyear(col("input")).alias("weekofyear") 
  ).show()

+----------+----+-----+----------+----------+
|     input|year|month|  next_day|weekofyear|
+----------+----+-----+----------+----------+
|2020-02-01|2020|    2|2020-02-02|         5|
|2019-03-11|2019|    3|2019-03-17|        11|
|2021-03-28|2021|    3|2021-04-04|        12|
+----------+----+-----+----------+----------+



#### dayofweek(), dayofmonth(), dayofyear()

In [11]:
df.select(col("input"),  
     dayofweek(col("input")).alias("dayofweek"), 
     dayofmonth(col("input")).alias("dayofmonth"), 
     dayofyear(col("input")).alias("dayofyear"), 
  ).show()

+----------+---------+----------+---------+
|     input|dayofweek|dayofmonth|dayofyear|
+----------+---------+----------+---------+
|2020-02-01|        7|         1|       32|
|2019-03-11|        2|        11|       70|
|2021-03-28|        1|        28|       87|
+----------+---------+----------+---------+



In [12]:
data=[["1","02-01-2020 11 01 19 06"],["2","03-01-2019 12 01 19 406"],["3","03-01-2021 12 01 19 406"]]
df2=spark.createDataFrame(data,["id","input"])
df2.printSchema()
df2.show(truncate=False)

root
 |-- id: string (nullable = true)
 |-- input: string (nullable = true)

+---+-----------------------+
|id |input                  |
+---+-----------------------+
|1  |02-01-2020 11 01 19 06 |
|2  |03-01-2019 12 01 19 406|
|3  |03-01-2021 12 01 19 406|
+---+-----------------------+



#### current_timestamp()
returns the current timestamp in spark default format yyyy-MM-dd HH:mm:ss

In [13]:
#current_timestamp()
df2.select(current_timestamp().alias("current_timestamp")
  ).show(1,truncate=False)

+-----------------------+
|current_timestamp      |
+-----------------------+
|2023-03-07 19:50:41.001|
+-----------------------+
only showing top 1 row



#### to_timestamp()
Converts string timestamp to Timestamp type format.

In [14]:
#to_timestamp()
df2.select(col("input"), 
    to_timestamp(col("input"), "MM-dd-yyyy HH mm ss SSS").alias("to_timestamp") 
  ).show(truncate=False)

+-----------------------+-----------------------+
|input                  |to_timestamp           |
+-----------------------+-----------------------+
|02-01-2020 11 01 19 06 |2020-02-01 11:01:19.06 |
|03-01-2019 12 01 19 406|2019-03-01 12:01:19.406|
|03-01-2021 12 01 19 406|2021-03-01 12:01:19.406|
+-----------------------+-----------------------+



#### hour(), Minute() and second()

In [15]:
#hour, minute,second
data=[["1","2020-02-01 11:01:19.06"],["2","2019-03-01 12:01:19.406"],["3","2021-03-01 12:01:19.406"]]
df3=spark.createDataFrame(data,["id","input"])

df3.select(col("input"), 
    hour(col("input")).alias("hour"), 
    minute(col("input")).alias("minute"),
    second(col("input")).alias("second") 
  ).show(truncate=False)

+-----------------------+----+------+------+
|input                  |hour|minute|second|
+-----------------------+----+------+------+
|2020-02-01 11:01:19.06 |11  |1     |19    |
|2019-03-01 12:01:19.406|12  |1     |19    |
|2021-03-01 12:01:19.406|12  |1     |19    |
+-----------------------+----+------+------+

